In [1]:
# Importing libraries

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from time import sleep
from pymongo import MongoClient
import chromedriver_binary
from selenium import webdriver
from selenium.webdriver.common import by
from selenium.webdriver.common import action_chains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import select
import re
%matplotlib inline

In [99]:
# Setting up the selenium python webdriver

driver = webdriver.Chrome()
url = 'https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa,Residential-Plot&cityName=Bangalore'
driver.get(url)
driver.maximize_window()
sleep(3)

In [23]:
# Setting up of Mongodb connection
# Using Mongodb to ensure db remains intact and stored information is secure in case of code failure

client = MongoClient()
db_assignment2 = client['Data_Science_Unplan']                            # Database name  
collection_magicbricks = db_assignment2['Collection_Magicbricks_Unplan']  # Collection name under the above db

### Scraping Algorithm in brief :

In [ ]:
# ALGORITHM FOR SCRAPING :

# OPEN GIVEN MAIN URL 
# COUNT_PROPERTY = 0
# LOOP :
        
        # FIND ALL VISIBLE LINKS AT THE MOMENT
            # FOR EACH OF THE ABOVE LINK LOOP:
                    # IF EXISTS IN DB :
                            # MOVE TO NEXT LINK
                    
                    # ELSE :
                            # SWITCH TO THIS PROPERTY TAB
                            # OPEN SEPARATELY AND SCRAPE THE DATA ONLY WHEN EVIDENCE IS FOUND FOR THAT FIELD
                            
        # COUNT_PROPERTY = COUNT_PROPERTY + ALL VISIBLE LINKS THAT HAVE BEEN COVERED TILL NOW  
        # SWITCH TO MAIN_URL PAGE
        # SCROLL DOWN AS RELEVANT FOR NEW VISIBLE CONTENT  
            
        # IF COUNT_PROPERTY >= 500:
            # WE ARE DONE AND WE BREAK FROM OUTERMOST LOOP
                    

In [ ]:
# Universal Harvesting Loop :

count_for_property = 0  # Keep track of the number of properties we have covered in our DB (Target = 500)

while (True):
     
    all_linksPossible_AND_Visible_atCurrent = driver.find_elements_by_xpath(r'//div[contains(@id, "resultBlockWrapper")]//h3//a')
        
    for each_current_link in all_linksPossible_AND_Visible_atCurrent:
        
        dict_property_parameters = {}     # This dictionary shall contain all the wanted parameters for a property

        if collection_magicbricks.find_one({'Property URL':each_current_link.get_attribute('href')}):
            continue  # The property has been already scraped and is present inside our DB's Collection
        
        else:  # We need to open the property and scrape the data for the first time
            
            try:
                each_current_link.click()   # Click on the link to property present inside the box on the main give URL page
            except:
                continue         # For incase of defective or unresponsive links
                
                
            # Function call to make elements visible on PropertyTabPage
            
            Scroll_For_Making_Elements_Appear_And_Visible_OnPropertyPageTab(driver)  
            
            dict_property_parameters['Property URL'] = each_current_link.get_attribute('href')   # Property URL
            sleep(5)                   # Wait for 5 seconds till the new property url opens in a separate tab

            
            # Since we are on a new page, hence we need to change the driver window handle to that page 
            
            allWindowHandlesExisting = driver.window_handles
            parentWindowHandle = driver.current_window_handle

            for each_window_handle in allWindowHandlesExisting :
                if each_window_handle != parentWindowHandle:
                    driver.switch_to_window(each_window_handle)   # Switch to the property page tab that opened separately

            # Headline :

            try:        
                Headline = driver.find_element_by_xpath(r"//div[@class='propBhk']//h1//span").text.strip()
                dict_property_parameters['Headline'] = Headline

            except:
                dict_property_parameters['Headline'] = None

            # Price :

            try:
                Price = driver.find_element_by_xpath(r"//div[@id='priceSv']").text.strip()
                dict_property_parameters['Price'] = Price 

            except:
                dict_property_parameters['Price'] = None

            # Owner/Builder :

            try:    
                Owner_Builder = driver.find_element_by_xpath(r"//div[@class='agentName']//div[@class='nameValue']").text.strip()
                dict_property_parameters['Owner_Builder'] = Owner_Builder

            except:
                dict_property_parameters['Owner_Builder'] = None

            # Description :

            try:
                Description = driver.find_element_by_xpath(r"//div[@class='propDescription clearfix']//div[@class='descriptionCont']//div[@class='p_infoRow'][1]").text.strip()  
                dict_property_parameters['Description'] = re.sub(r"\n|more",'',str(Description))

            except:
                dict_property_parameters['Description'] = None


            #------------------------------------------------------------------------------------------------
            # Bedrooms,  Bathrooms, Balconies :
            
            # We tried to extract maximum parameters but we don't know the the number of above parameters we have obtained 
            # hence, we perform a check on the retrieved data and only those found are added to our major data containing dictionary
            # The same goes for other property parameters/fields below :
            
            dict_bathroom_bedroom_balcony = getBedrooms_AND_Bathrooms_AND_BalconyValueIfExists(driver) # Fn to get bathrooms,bedrooms, balcony  
            
            if 'Bedrooms' in dict_bathroom_bedroom_balcony.keys():
                dict_property_parameters['Bedrooms'] = dict_bathroom_bedroom_balcony['Bedrooms']
            else:
                dict_property_parameters['Bedrooms'] = None
            
            if 'Bathrooms' in dict_bathroom_bedroom_balcony.keys():
                dict_property_parameters['Bathrooms'] = dict_bathroom_bedroom_balcony['Bathrooms']
            else:
                dict_property_parameters['Bathrooms'] = None
            
            if 'Balcony' in dict_bathroom_bedroom_balcony.keys():
                dict_property_parameters['Balcony'] = dict_bathroom_bedroom_balcony['Balcony']
            else:
                dict_property_parameters['Balcony'] = None
                
            #-------------------------------------------------------------------------------------------------

            
            #-------------------------------------------------------------------------------------------------
            # SuperArea, Price per sqft :
            
            dict_SuperArea_PricePersqft = getSuperArea_AND_PricePersqftValueifExists(driver)
            
            if 'Super Area' in dict_SuperArea_PricePersqft.keys():
                dict_property_parameters['Super Area'] = dict_SuperArea_PricePersqft['Super Area']
            else:
                dict_property_parameters['Super Area'] = None
            
            if 'Price per sqft' in dict_SuperArea_PricePersqft.keys():
                dict_property_parameters['Price per sqft'] = dict_SuperArea_PricePersqft['Price per sqft']
            else:
                dict_property_parameters['Price per sqft'] = None
            
            #-------------------------------------------------------------------------------------------------
            
            
            #-------------------------------------------------------------------------------------------------
            # Status, TransactionType, Floor, Car Parking :

            dict_Status_TransactionType_Floor_CarParking = getStatus_AND_TransactionType_AND_Floor_AND_CarParkingValueifExists(driver)
            
            if 'Status' in dict_Status_TransactionType_Floor_CarParking.keys():
                dict_property_parameters['Status'] = dict_Status_TransactionType_Floor_CarParking['Status']
            else:
                dict_property_parameters['Status'] = None
            
            if 'Transaction type' in dict_Status_TransactionType_Floor_CarParking.keys():
                dict_property_parameters['Transaction type'] = dict_Status_TransactionType_Floor_CarParking['Transaction type']
            else:
                dict_property_parameters['Transaction type'] = None
            
            if 'Floor' in dict_Status_TransactionType_Floor_CarParking.keys():
                dict_property_parameters['Floor'] = dict_Status_TransactionType_Floor_CarParking['Floor']
            else:
                dict_property_parameters['Floor'] = None
            
            if 'Car parking' in dict_Status_TransactionType_Floor_CarParking.keys():
                dict_property_parameters['Car parking'] = dict_Status_TransactionType_Floor_CarParking['Car parking']
            else:
                dict_property_parameters['Car parking'] = None
            
            #--------------------------------------------------------------------------------------------------

            #--------------------------------------------------------------------------------------------------
            # Furnished status, Lifts :
            
            dict_FurnishedStatus_AND_LiftValue = getFurnishedStatus_AND_LiftValueifExists(driver)
            
            if 'Furnished status' in dict_FurnishedStatus_AND_LiftValue.keys():
                dict_property_parameters['Furnished status'] = dict_FurnishedStatus_AND_LiftValue['Furnished status']
            else:
                dict_property_parameters['Furnished status'] = None
            
            if 'Lifts' in dict_FurnishedStatus_AND_LiftValue.keys():
                dict_property_parameters['Lifts'] = dict_FurnishedStatus_AND_LiftValue['Lifts']
            else:
                dict_property_parameters['Lifts'] = None
            
            #--------------------------------------------------------------------------------------------------
            
        
        
            #--------------------------------------------------------------------------------------------------
            # PriceBreakup, Address, Landmarks, AgeofConstruction :

            dict_PriceBreakup_Address_Landmarks_AgeOfConstruction = getPriceBreakup_AND_Address_AND_Landmarks_AND_AgeOfConstructionValueifExists(driver)
            
            if 'Price Breakup' in dict_PriceBreakup_Address_Landmarks_AgeOfConstruction.keys():
                dict_property_parameters['Price Breakup'] = dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Price Breakup']
            else:
                dict_property_parameters['Price Breakup'] = None
            
            if 'Address' in dict_PriceBreakup_Address_Landmarks_AgeOfConstruction.keys():
                dict_property_parameters['Address'] = dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Address']
            else:
                dict_property_parameters['Address'] = None
            
            if 'Age of Construction' in dict_PriceBreakup_Address_Landmarks_AgeOfConstruction.keys():
                dict_property_parameters['Age of Construction'] = dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Age of Construction']
            else:
                dict_property_parameters['Age of Construction'] = None
            
            if 'Landmarks' in dict_PriceBreakup_Address_Landmarks_AgeOfConstruction.keys():
                dict_property_parameters['Landmarks'] = dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Landmarks']
            else:
                dict_property_parameters['Landmarks'] = None
            
            #-------------------------------------------------------------------------------------------------

            #-------------------------------------------------------------------------------------------------
            # PriceComparision, MonthlyEMI, ExpectedRent :
            
            dict_PriceComparision_MonthlyEMI_ExpectedRent = getPriceComparision_AND_MonthlyEMI_AND_ExpectedRentValueifExists(driver)
            
            if 'Price comparison' in dict_PriceComparision_MonthlyEMI_ExpectedRent.keys():
                dict_property_parameters['Price comparison'] = dict_PriceComparision_MonthlyEMI_ExpectedRent['Price comparison'] 
            else:
                dict_property_parameters['Price comparison'] = None
                
            if 'Expected rent' in dict_PriceComparision_MonthlyEMI_ExpectedRent.keys():
                dict_property_parameters['Expected rent'] = dict_PriceComparision_MonthlyEMI_ExpectedRent['Expected rent']
            else:
                dict_property_parameters['Expected rent'] = None
                
            if 'Monthly EMI' in dict_PriceComparision_MonthlyEMI_ExpectedRent.keys():
                dict_property_parameters['Monthly EMI'] = dict_PriceComparision_MonthlyEMI_ExpectedRent['Monthly EMI']
            else:
                dict_property_parameters['Monthly EMI'] = None
            
            #------------------------------------------------------------------------------------------------
            
            # Inserting into collection inside mongodb since our containing dictionary has been populated as required
            collection_magicbricks.insert(dict_property_parameters)
            
            # Close this property current page
            driver.close()
            
            # Switch back to parent home page
            driver.switch_to_window(parentWindowHandle)
        
    # Count to keep track of the number of properties we have covered already
    
    count_for_property = count_for_property + len(all_linksPossible_AND_Visible_atCurrent)  
    
    # When back to the main URL, we scroll as required to make new boxes appear 
    
    Main_Page_PropertyBoxes_Scroll_Towards_Infinite(driver)  
    
    if count_for_property >= 500:    # Property limit given by default 
    #  print("Reached 500 already")
        break      

In [103]:
def Main_Page_PropertyBoxes_Scroll_Towards_Infinite(driver):
    '''
    When we are on the main page URL, that contains property boxes, so once we have covered the already set of visible
    properties by iterating or operning, through them each, individually and separately and once this set is over, 
    Then we switch back to this main URL Page and scroll towards END and back so as to make new property boxes appear 
    and visible and then we start to iterate through them separately and reach end and back and continue this procedure 
    till 500 genuine properties are covered inside our Mongo Collection
    '''
    
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    driver.implicitly_wait(5)
    driver.execute_script('window.scrollBy(0, -300);')
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    driver.implicitly_wait(2)

In [104]:
def Scroll_For_Making_Elements_Appear_And_Visible_OnPropertyPageTab(driver):
    '''
    This function scrolls about 12 times and each scroll has magnitude of 300 selenium units downwards.
    This is necessary for making the desired elements visible on the property page and reduces the risk of missing the 
    elements while scraping.
    '''
    scrollCount = 0
    while (True):
        driver.execute_script('window.scrollBy(0, 300);')    
        scrollCount += 1
        sleep(1)

        if scrollCount == 12:
            driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
            break

In [9]:
def getBedrooms_AND_Bathrooms_AND_BalconyValueIfExists(driver):
    '''
    This function scrapes the value for Bedrooms and Bathrooms by finding the existence for the same at first and if 
    existence is found, then only scrape the respective values.
    '''
    sleep(2)
    firstRowFoldElementsTotal = len(driver.find_elements_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='firstFoldDisplay']//div[@class='p_title']"))
    dict_bathroom_bedroom_balcony = {}   # dictionary to hold no of bedrooms, bathrooms and balconies
    
    for j in range(1, firstRowFoldElementsTotal + 1):
        category = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='firstFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[@class='p_title']").text

        if len(re.findall(r"Bedroom", str(category.strip()))) > 0:    # Evidence of bedroom
        
            try:
                noOfBedrooms = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='firstFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text.strip()  
                dict_bathroom_bedroom_balcony['Bedrooms'] = noOfBedrooms
            except:
                dict_bathroom_bedroom_balcony['Bedrooms'] = None
                
        
        if len(re.findall(r"Bathroom", str(category.strip()))) > 0:  # Evidence of bathroom
            
            try:
                noOfBathrooms = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='firstFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_bathroom_bedroom_balcony['Bathrooms'] = noOfBathrooms
            except:
                dict_bathroom_bedroom_balcony['Bathrooms'] = None
                
        if len(re.findall(r"Balcon", str(category.strip()))) > 0: # Evidence of balconies
            try:
                noOfBalcony = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='firstFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_bathroom_bedroom_balcony['Balcony'] = noOfBalcony
            except:
                dict_bathroom_bedroom_balcony['Balcony'] = None
            
            
    return dict_bathroom_bedroom_balcony
    

In [10]:
def getSuperArea_AND_PricePersqftValueifExists(driver):
    '''
    This function scrapes the value for SuperArea and Price per sqft by finding the existence for the same at first and if 
    existence is found, then only scrape the respective values.
    '''
    sleep(2)
    secondRowFoldElementsTotal = len(driver.find_elements_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='secondFoldDisplay']//div[@class='p_title']"))
    dict_SuperArea_PricePersqft = {}
    
    for j in range(1, secondRowFoldElementsTotal + 1):
        category = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='secondFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[@class='p_title']").text
        
        if category.strip() == 'Super area':
            try:
                valueSuperArea = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='secondFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_SuperArea_PricePersqft['Super Area'] = valueSuperArea
            except:
                dict_SuperArea_PricePersqft['Super Area'] = None
                
                
            try:
                valuePricePerSqft = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='secondFoldDisplay']//div[@class='p_infoColumn']//div[@class='fo_11px c_dark_gray']").text
                dict_SuperArea_PricePersqft['Price per sqft'] = valuePricePerSqft  
            except:
                dict_SuperArea_PricePersqft['Price per sqft'] = None
                
    return dict_SuperArea_PricePersqft

In [11]:
def getStatus_AND_TransactionType_AND_Floor_AND_CarParkingValueifExists(driver):
    '''
    This function scrapes the value for Status AND TransactionType AND Floor AND CarParking Value by finding the existence 
    for the same at first and if existence is found, then only scrape the respective values.
    '''
    sleep(2)
    fourthRowFoldElementsTotalCount = len(driver.find_elements_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='fourthFoldDisplay']//div[@class='p_title']"))
    dict_Status_TransactionType_Floor_CarParking = {}
    
    for j in range(1, fourthRowFoldElementsTotalCount + 1):
        category = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='fourthFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[@class='p_title']").text
        
        if category.strip() == 'Status':
            try:
                valueStatus = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='fourthFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_Status_TransactionType_Floor_CarParking['Status'] = valueStatus
            except:
                dict_Status_TransactionType_Floor_CarParking['Status'] = None
        
        if category.strip() == 'Transaction type':
            try:
                valueTransactionType = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='fourthFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_Status_TransactionType_Floor_CarParking['Transaction type'] = valueTransactionType
            except:
                dict_Status_TransactionType_Floor_CarParking['Transaction type'] = None
        
        if category.strip() == 'Floor':
            try:
                valueFloor = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='fourthFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_Status_TransactionType_Floor_CarParking['Floor'] = valueFloor
            except:
                dict_Status_TransactionType_Floor_CarParking['Floor'] = None
            
        
        if category.strip() == 'Car parking':
            try:
                valueCarParking = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='fourthFoldDisplay']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_Status_TransactionType_Floor_CarParking['Car parking'] = valueCarParking
            except:
                dict_Status_TransactionType_Floor_CarParking['Car parking'] = None
                
    return dict_Status_TransactionType_Floor_CarParking
            

In [12]:
def getFurnishedStatus_AND_LiftValueifExists(driver):   
    '''
    This function scrapes the value for FurnishedStatus AND LiftValue Value by finding the existence for the same at first 
    and if existence is found, then only scrape the respective values.
    '''
    sleep(2)     
    forthfoldAllRegCounterExtElementsTotalCount = len(driver.find_elements_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='forthfoldAllRegCounterExt']//div[@class='p_title']"))
    dict_FurnishedStatus_AND_LiftValue = {}
    
    for j in range(1, forthfoldAllRegCounterExtElementsTotalCount + 1):
        category = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='forthfoldAllRegCounterExt']//div[@class='p_infoColumn'][" + str(j) + "]//div[@class='p_title']").text
        
        if category.strip() == 'Furnished status':
            try:
                valueFurnishedStatus = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='forthfoldAllRegCounterExt']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_FurnishedStatus_AND_LiftValue['Furnished status'] = valueFurnishedStatus
            except:
                dict_FurnishedStatus_AND_LiftValue['Furnished status'] = None

        if category.strip() == 'Lifts':
            try:
                valueLifts = driver.find_element_by_xpath(r"//div[@class='propInfoBlockInn']//div[@id='forthfoldAllRegCounterExt']//div[@class='p_infoColumn'][" + str(j) + "]//div[contains(@class,'p_value')]").text  
                dict_FurnishedStatus_AND_LiftValue['Lifts'] = valueLifts
            except:
                dict_FurnishedStatus_AND_LiftValue['Lifts'] = None
                
    return dict_FurnishedStatus_AND_LiftValue

In [13]:
def getPriceBreakup_AND_Address_AND_Landmarks_AND_AgeOfConstructionValueifExists(driver):
    '''
    This function scrapes the value for PriceBreakup AND Address AND Landmarks AND AgeOfConstruction by finding the existence for the same at first 
    and if existence is found, then only scrape the respective values.
    '''
    sleep(2)
    driver.execute_script("window.scrollBy(0, 400);")  # scrolling a while towards bottom
    # Items are present inside 'descriptionCont' container

    total_elements_count_descriptionCont = driver.find_elements_by_xpath(r"//div[@class='propDescription clearfix']//div[@class='descriptionCont']//div[@class='p_infoRow']//div[@class='p_title']")
    dict_PriceBreakup_Address_Landmarks_AgeOfConstruction = {}

    for k in range(2, len(total_elements_count_descriptionCont) + 2):  

        category = driver.find_element_by_xpath(r"//div[@class='propDescription clearfix']//div[@class='descriptionCont']//div[@class='p_infoRow']["+str(k)+"]//div[@class='p_title']").text
        
        if category == 'Price Breakup':
            try:
                valuePriceBreakup = driver.find_element_by_xpath(r"//div[@class='propDescription clearfix']//div[@class='descriptionCont']//div[@class='p_infoRow'][" + str(k) + "]//div[contains(@class,'p_value')]").text
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Price Breakup'] = valuePriceBreakup 
            except:
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Price Breakup'] = None
                
        if category == 'Address':
            try:
                valueAddress = driver.find_element_by_xpath(r"//div[@class='propDescription clearfix']//div[@class='descriptionCont']//div[@class='p_infoRow'][" + str(k) + "]//div[contains(@class,'p_value')]").text
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Address'] = valueAddress
            except:
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Address'] = None
            
        if category == 'Landmarks':
            try:
                valueLandmarks = driver.find_element_by_xpath(r"//div[@class='propDescription clearfix']//div[@class='descriptionCont']//div[@class='p_infoRow'][" + str(k) + "]//div[contains(@class,'p_value')]").text
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Landmarks'] = valueLandmarks
            except:
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Landmarks'] = None
                
        if category == 'Age of Construction':
            try:
                valueAgeOfConstruction = driver.find_element_by_xpath(r"//div[@class='propDescription clearfix']//div[@class='descriptionCont']//div[@class='p_infoRow'][" + str(k) + "]//div[contains(@class,'p_value')]").text
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Age of Construction'] = valueAgeOfConstruction
            except:
                dict_PriceBreakup_Address_Landmarks_AgeOfConstruction['Age of Construction'] = None
                
    return dict_PriceBreakup_Address_Landmarks_AgeOfConstruction

In [14]:
def getPriceComparision_AND_MonthlyEMI_AND_ExpectedRentValueifExists(driver):
    '''
    This function scrapes the value for PriceComparision MonthlyEMI ExpectedRent by finding the existence for the same at first 
    and if existence is found, then only scrape the respective values.
    '''
    sleep(2)
    driver.execute_script("window.scrollBy(0, 600);")  # scrolling a while towards bottom
    total_elements_count_underQuickFacts = len(driver.find_elements_by_xpath(r"//div[@id='quickFactsOnProperty']//div[@class='quick_row']//div[@class='column']"))
    dict_PriceComparision_MonthlyEMI_ExpectedRent = {}
    
    for n in range(1, total_elements_count_underQuickFacts + 1):
        category = driver.find_element_by_xpath(r"//div[@id='quickFactsOnProperty']//div[@class='quick_row']//div[@class='column'][" + str(n) + "]//div[@class='pc_label']").text
    
        if category.strip() == 'Price comparison':
            try:
                valuePriceComparision = driver.find_element_by_xpath(r"//div[@id='quickFactsOnProperty']//div[@class='quick_row']//div[@class='column'][" + str(n) + "]//div[@class='pc_value']").text.strip()
                dict_PriceComparision_MonthlyEMI_ExpectedRent['Price comparison'] = valuePriceComparision
            except:
                dict_PriceComparision_MonthlyEMI_ExpectedRent['Price comparison'] = None
            
            
        if category.strip() == 'Monthly EMI':
            try:
                valueMonthlyEMI = driver.find_element_by_xpath(r"//div[@id='quickFactsOnProperty']//div[@class='quick_row']//div[@class='column'][" + str(n) + "]//div[@class='pc_value']").text.strip()
                dict_PriceComparision_MonthlyEMI_ExpectedRent['Monthly EMI'] = valueMonthlyEMI
            except:
                dict_PriceComparision_MonthlyEMI_ExpectedRent['Monthly EMI'] = None
            

        if category.strip() == 'Expected rent':
            try:
                valueExpectedRent = driver.find_element_by_xpath(r"//div[@id='quickFactsOnProperty']//div[@class='quick_row']//div[@class='column'][" + str(n) + "]//div[@class='pc_value']").text.strip()
                dict_PriceComparision_MonthlyEMI_ExpectedRent['Expected rent'] = valueExpectedRent
            except:
                dict_PriceComparision_MonthlyEMI_ExpectedRent['Expected rent'] = None
                
    return dict_PriceComparision_MonthlyEMI_ExpectedRent